In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

sys.path.append('/home/mei/Documents/deep_sort/exp/20220609/utils')

from utils.frame import sec2frame
from utils.load_data import (load_annotated_trackret_file,
                             get_player_id_list,
                             load_pose_data,
                             load_bbox_data
                            )
from utils.matching import match_id_by_iou
from utils.frame import frame2sec, groupby_rally
from utils.func import multiprocess_imap
from IPython.core.debugger import Pdb
import multiprocessing
from multiprocessing import Manager, Process
from tqdm.notebook import tqdm
import pickle
import subprocess

In [2]:
ist = Pdb().set_trace

In [3]:
data_file_path = Path(os.getcwd())/'data'
data_name = data_file_path / 'p008_wtnb_0528.csv'
output_file_path: str = Path(os.getcwd())/'output'

In [4]:
df = pd.read_csv(data_name)
df.head()

,start,end,labels
0,77.761265,77.861253,OA
1,78.395459,78.470450,XB
2,78.978720,79.045378,XA
3,79.561053,79.602715,OB
4,91.541977,91.583639,OA


In [14]:
df = groupby_rally(df)
input_file = str(output_file_path/'output.mp4')
group_ids = df.groupby('group').groups
margin = 2

In [18]:
def wrap(i):
    tar_ids = group_ids[i]
    frame_df = df.iloc[tar_ids,:]
    end_time:float = frame2sec(frame_df['start_frame'].max(),fps=25) + margin
    start_time:float = frame2sec(frame_df['start_frame'].min(),fps=25) - margin
    v_frames = sec2frame(end_time-start_time,fps=25)
    output_file = str(output_file_path/f'{i}.mp4')
    # if Path(output_file).exists():
    #     # print(f'{output_file} is exist')
    #     return 
    command = command = f'ffmpeg -y -ss {start_time} -i {input_file} -frames {v_frames}  {output_file}'

    subprocess.call(command, shell=True)

In [19]:
multiprocess_imap(wrap, range(len(group_ids)))